<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Observability for Maltbot / Clawdbot with Langfuse" sidebarTitle: "Maltbot / Clawdbot" logo: "/images/integrations/opentelemetry_icon.svg" description: "Learn how to trace Maltbot and Clawdbot interactions using Langfuse via OpenTelemetry." category: "Integrations" -->

# Integrate Langfuse with Maltbot / Clawdbot

This notebook demonstrates how to integrate **Langfuse** with **Maltbot** (also known as **Clawdbot**) using OpenTelemetry. By the end of this guide, you will be able to trace your Maltbot interactions with Langfuse for improved observability and debugging.

> **What is Maltbot / Clawdbot?** [Maltbot](https://molt.bot/) (Clawdbot) is an AI-powered conversational bot framework that exposes OpenTelemetry spans for detailed diagnostics and observability. It allows developers to build intelligent conversational agents while maintaining full visibility into the bot's reasoning and decision-making processes.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open-source LLM engineering platform. It provides tracing and monitoring capabilities for LLM applications, helping developers debug, analyze, and optimize their AI systems. Langfuse integrates with various tools and frameworks via native integrations, OpenTelemetry, and API/SDKs.

## How it Works

Maltbot exposes [OpenTelemetry spans](https://docs.molt.bot/logging#diagnostics-+-opentelemetry) that capture detailed information about bot interactions, including:

- Conversation turns and messages
- LLM calls and responses
- Tool invocations and results
- Reasoning steps and decisions

Langfuse operates as an [OpenTelemetry Backend](/docs/opentelemetry/get-started) and receives these spans on the `/api/public/otel` endpoint. This integration allows you to leverage Langfuse's powerful tracing UI to analyze and debug your Maltbot conversations.

## Get Started

We'll walk through the setup and configuration to send Maltbot traces to Langfuse.

<!-- STEPS_START -->
### Step 1: Install Dependencies

In [ ]:
%pip install maltbot opentelemetry-sdk opentelemetry-exporter-otlp langfuse

### Step 2: Set Up Environment Variables

Get your Langfuse API keys by signing up for [Langfuse Cloud](https://cloud.langfuse.com) or [self-hosting Langfuse](https://langfuse.com/self-hosting).

In [ ]:
import os
import base64

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_BASE_URL"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_BASE_URL"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

With the environment variables set, we can now initialize the Langfuse client. `get_client()` initializes the Langfuse client using the credentials provided in the environment variables.

In [ ]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

### Step 3: Configure OpenTelemetry to Export to Langfuse

Maltbot uses OpenTelemetry for tracing. We need to configure the OpenTelemetry SDK to export traces to Langfuse's OTel endpoint.

First, set up the OTel exporter with Langfuse authentication:

In [ ]:
# Configure OTel endpoint and authentication for Langfuse
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_BASE_URL") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

Now, configure the OpenTelemetry TracerProvider with an exporter that sends traces to Langfuse:

In [ ]:
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry import trace

# Create and configure the tracer provider
tracer_provider = TracerProvider()

# Add a batch span processor with the OTLP exporter
# OTLPSpanExporter() uses the endpoint and headers from environment variables
tracer_provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter()))

# Set the global tracer provider
trace.set_tracer_provider(tracer_provider)

print("OpenTelemetry configured to export traces to Langfuse!")

### Step 4: Initialize Maltbot with Tracing Enabled

Now initialize Maltbot with OpenTelemetry tracing enabled. Maltbot will automatically emit spans that get exported to Langfuse.

In [ ]:
from maltbot import Maltbot, MaltbotConfig

# Configure Maltbot with OpenTelemetry tracing enabled
config = MaltbotConfig(
    enable_telemetry=True,  # Enable OpenTelemetry tracing
    telemetry_service_name="maltbot-langfuse-example",
)

# Initialize the bot
bot = Maltbot(config=config)

print("Maltbot initialized with tracing enabled!")

### Step 5: Run a Conversation and Generate Traces

Now let's run a conversation with Maltbot. The OpenTelemetry spans will be automatically sent to Langfuse.

In [ ]:
# Run a sample conversation
response = bot.chat(
    message="What are the key benefits of using observability tools for AI applications?",
    session_id="demo-session-001",
    user_id="demo-user"
)

print("Bot response:")
print(response)

Let's run a more complex multi-turn conversation to see richer traces:

In [ ]:
# Multi-turn conversation example
session_id = "demo-session-002"

# First turn
response1 = bot.chat(
    message="I'm building an AI chatbot. What should I consider for production deployment?",
    session_id=session_id,
    user_id="demo-user"
)
print("Turn 1 response:", response1[:200] + "..." if len(response1) > 200 else response1)

# Second turn - follow-up question
response2 = bot.chat(
    message="Can you elaborate on monitoring and observability specifically?",
    session_id=session_id,
    user_id="demo-user"
)
print("\nTurn 2 response:", response2[:200] + "..." if len(response2) > 200 else response2)

# Third turn - specific request
response3 = bot.chat(
    message="What metrics should I track to measure my chatbot's performance?",
    session_id=session_id,
    user_id="demo-user"
)
print("\nTurn 3 response:", response3[:200] + "..." if len(response3) > 200 else response3)

### Step 6: Flush Traces and View in Langfuse

Ensure all traces are sent to Langfuse by flushing the tracer provider:

In [ ]:
# Flush all pending spans to ensure they're exported
tracer_provider.force_flush()

print("All traces have been sent to Langfuse!")
print(f"View your traces at: {os.environ.get('LANGFUSE_BASE_URL')}")

### Step 7: View Traces in Langfuse

After running the bot interactions above, navigate to your [Langfuse dashboard](https://cloud.langfuse.com) to view the traces. You'll see detailed information about each conversation turn, including:

- **Conversation flow**: The sequence of messages and responses
- **LLM calls**: Model inputs, outputs, and token usage
- **Latencies**: Time taken for each step in the conversation
- **Session grouping**: Related conversations grouped by session ID

![Maltbot example trace](https://langfuse.com/images/cookbook/integration-maltbot/maltbot-example-trace.png)
<!-- STEPS_END -->

## Adding Custom Attributes

You can add custom attributes to your traces using Langfuse-specific OTel attributes. These allow you to filter and search traces more effectively.

In [ ]:
# Get a tracer for custom spans
tracer = trace.get_tracer("maltbot-custom")

# Create a custom span with Langfuse attributes
with tracer.start_as_current_span("custom-maltbot-interaction") as span:
    # Set Langfuse-specific attributes for better filtering
    span.set_attribute("langfuse.user.id", "user-123")
    span.set_attribute("langfuse.session.id", "session-456")
    span.set_attribute("langfuse.trace.tags", ["production", "customer-support"])
    span.set_attribute("langfuse.trace.metadata.customer_tier", "premium")
    span.set_attribute("langfuse.trace.metadata.bot_version", "1.2.0")
    
    # Run the bot within this span context
    response = bot.chat(
        message="Help me understand my billing statement.",
        session_id="session-456",
        user_id="user-123"
    )
    
    # Set the output on the span
    span.set_attribute("langfuse.trace.output", response)

print("Response:", response)

## Alternative: Using Environment Variables

Instead of configuring OTel in code, you can set environment variables before starting your application. This is useful for production deployments:

```bash
# Set Langfuse credentials
export LANGFUSE_PUBLIC_KEY="pk-lf-..."
export LANGFUSE_SECRET_KEY="sk-lf-..."
export LANGFUSE_BASE_URL="https://cloud.langfuse.com"

# Generate the auth string
export LANGFUSE_AUTH=$(echo -n "${LANGFUSE_PUBLIC_KEY}:${LANGFUSE_SECRET_KEY}" | base64)

# Configure OTel to export to Langfuse
export OTEL_EXPORTER_OTLP_ENDPOINT="${LANGFUSE_BASE_URL}/api/public/otel"
export OTEL_EXPORTER_OTLP_HEADERS="Authorization=Basic ${LANGFUSE_AUTH}"

# Enable Maltbot telemetry
export MALTBOT_ENABLE_TELEMETRY="true"
export MALTBOT_TELEMETRY_SERVICE_NAME="my-maltbot-service"

# Run your application
python my_maltbot_app.py
```

## Using with OpenTelemetry Collector

For more advanced setups, you can use an [OpenTelemetry Collector](https://opentelemetry.io/docs/collector/) to process and route traces. This is useful when you want to send traces to multiple backends or apply transformations.

Example Collector configuration to forward Maltbot traces to Langfuse:

```yaml
receivers:
  otlp:
    protocols:
      grpc:
        endpoint: 0.0.0.0:4317
      http:
        endpoint: 0.0.0.0:4318

processors:
  batch:

exporters:
  otlphttp/langfuse:
    endpoint: "https://cloud.langfuse.com/api/public/otel"
    headers:
      Authorization: "Basic ${LANGFUSE_AUTH}"

service:
  pipelines:
    traces:
      receivers: [otlp]
      processors: [batch]
      exporters: [otlphttp/langfuse]
```

Then configure Maltbot to send traces to your Collector instead of directly to Langfuse.

## Learn More

- [Maltbot Documentation](https://docs.molt.bot/)
- [Langfuse OpenTelemetry Integration](/docs/opentelemetry/get-started)
- [Langfuse Attribute Mapping](/docs/opentelemetry/get-started#property-mapping)
- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/languages/python/)

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->